## 深度学习特征

提取CT、MRI、内镜、Xray等影像数据的深度学习特征。

### Onekey步骤

1. 将待提取的数据转化成jpg，可以参考使用OKT-convert2jpg或者OKT-crop_max_roi两个Onekey工具。
2. 获取到指定目录的所有图像数据。
3. 选择要提取什么样的模型的深度学习特征，目前Onekey支持主流的深度学习模型。（可以考虑使用Onekey进行迁移学习）
4. 提取特征，保存特征文件。

## 获取待提取特征的文件

提供两种批量处理的模式：
1. 目录模式，提取指定目录下的所有jpg文件的特征。
2. 文件模式，待提取的数据存储在文件中，每行一个样本。

当然也可以在最后自己指定手动提取指定若干文件。

In [ ]:
import os
# 目录模式
mydir = r'C:\Users\onekey\Project\OnekeyDS\CT\crop'
# mydir = r'C:\Users\onekey\Project\OnekeyDS\CT\full'
directory = os.path.expanduser(mydir)
test_samples = [os.path.join(directory, p) for p in os.listdir(directory) if p.endswith('.png') or p.endswith('.jpg')]

# 文件模式
# test_file = ''
# with open(test_file) as f:
#     test_samples = [l.strip() for l in f.readlines()]

# 自定义模式
# test_sampleses = ['path2jpg']
test_samples

## 确定提取特征

通过关键词获取要提取那一层的特征。

### 支持的模型名称

模型名称替换代码中的 `model_name`变量的值。

| **模型系列** | **模型名称**                                                 |
| ------------ | ------------------------------------------------------------ |
| AlexNet      | alexnet                                                      |
| VGG          | vgg11, vgg11_bn, vgg13, vgg13_bn, vgg16, vgg16_bn, vgg19_bn, vgg19 |
| ResNet       | resnet18, resnet34, resnet50, resnet101, resnet152, resnext50_32x4d, resnext101_32x8d, wide_resnet50_2, wide_resnet101_2 |
| DenseNet     | densenet121, densenet169, densenet201, densenet161           |
| Inception    | googlenet, inception_v3                                      |
| SqueezeNet   | squeezenet1_0, squeezenet1_1                                 |
| ShuffleNetV2 | shufflenet_v2_x2_0, shufflenet_v2_x0_5, shufflenet_v2_x1_0, shufflenet_v2_x1_5 |
| MobileNet    | mobilenet_v2, mobilenet_v3_large, mobilenet_v3_small         |
| MNASNet      | mnasnet0_5, mnasnet0_75, mnasnet1_0, mnasnet1_3              |

In [ ]:
from onekey_algo.custom.components.comp2 import extract, print_feature_hook, reg_hook_on_module, \
    init_from_model, init_from_onekey

model_name = 'resnet50'
model, transformer, device = init_from_model(model_name=model_name)
# model, transformer, device = init_from_onekey(r'')
for n, m in model.named_modules():
    print('Feature name:', n, "|| Module:", m)

## 提取特征

`Feature name:` 之后的名称为要提取的特征名，例如`layer3.0.conv2`, 一般深度学习特征提取最后一层，例如`avgpool`

In [ ]:
from functools import partial
feature_name = 'avgpool'
with open('feature.csv', 'w') as outfile:
    hook = partial(print_feature_hook, fp=outfile)
    find_num = reg_hook_on_module(feature_name, model, hook)
    results = extract(test_samples, model, transformer, device, fp=outfile)

## 读取数据

In [ ]:
import pandas as pd
features = pd.read_csv('feature.csv', header=None)
features.columns=['ID'] + list(features.columns[1:])
features.head()

### 深度特征压缩

深度学习特征压缩，注意压缩到的维度需要小于样本数

```python
def compress_df_feature(features: pd.DataFrame, dim: int, not_compress: Union[str, List[str]] = None,
                        prefix='') -> pd.DataFrame:
    """
    压缩深度学习特征
    Args:
        features: 特征DataFrame
        dim: 需要压缩到的维度，此值需要小于样本数
        not_compress: 不进行压缩的列。
        prefix: 所有特征的前缀。

    Returns:

    """
```

In [ ]:
from onekey_algo.custom.components.comp1 import compress_df_feature

cm_features = compress_df_feature(features=features, dim=32, prefix='DL_', not_compress='ID')
cm_features.to_csv('compress_features.csv', header=True, index=False)